In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
np.random.seed(42)
import pprint
import s3fs

import models, cross_validation, comparator

In [2]:
path = "s3://aifluence-data2/data/02_intermediate/chatelet_pdt/H.parq"
df = pd.read_parquet(path, engine="pyarrow")

# on a un csv avec multiindex "line", "station", "datetime", "direction"

NoCredentialsError: Unable to locate credentials

In [3]:
models_to_test = [
  models.ImputeByMean(["datetime.dt.hour", "datetime.dt.dayofweek", "direction"]), 
  models.ImputeByMedian(["datetime.dt.hour", "datetime.dt.dayofweek", "direction"]), 
  models.RandomImpute(),
  models.ImputeLOCF(["direction"]),
  models.ImputeNOCB(["direction"]),
  models.ImputeKNN(),
#   models.ImputeProphet(**{
#               "weekly_seasonality": True, 
#               "yearly_seasonality": True,
#               "interval_width": 0.95,
#               })
# models.ImputeRPCA(),
]

search_params = {
  "ImputeKNN": {"k": {"min":2, "max":3, "type":"Integer"}},
  "ImputeProphet": {
          "daily_seasonality": {"categories": [True, False], "type":"Categorical"},
        }
  }

line = "H"
station = "VALMONDOIS"
# dataset = df[(df["line"]==line) & (df["station"]==station)]
dataset = df.loc[line].loc[station]
cols_to_impute = ["load"]
dataset = dataset.iloc[:10]

# dataset = dataset.set_index(["line","station", "datetime", "direction"])
dataset = dataset.set_index(["datetime", "direction"])
# dataset = dataframe with multiindex and at least, columns in cols_to_impute

prop_nan = 0.5

In [4]:
dataset

,,train,position,date_count,n_coaches,mission,datetime_theo_cht,loading,unloading,load,origin_cht,destination_cht,ORIGIN,DESTINATION
datetime,direction,,,,,,,,,,,,,
NaT,Paris,121006,1,2019-01-01,1,None,NaT,NaN,NaN,NaN,None,None,PERSAN BEAUMONT,PONTOISE
2019-01-01 07:20:36,Suburb,121009,1,2019-01-01,1,TSOL,2019-01-01 07:18:30,0.0,0.0,6.0,Pontoise,Creil,PONTOISE,CREIL
2019-01-01 07:37:38,Paris,121012,1,2019-01-01,1,TOLI,2019-01-01 07:35:50,0.0,0.0,17.0,Creil,Pontoise,CREIL,PONTOISE
2019-01-01 08:19:10,Suburb,121015,1,2019-01-01,1,TSOL,2019-01-01 08:18:30,10.0,0.0,26.0,Pontoise,Creil,PONTOISE,CREIL
2019-01-01 08:37:00,Paris,121020,1,2019-01-01,1,TIMA,2019-01-01 08:36:10,1.0,5.0,13.0,Persan-Beaumont,Pontoise,PERSAN BEAUMONT,PONTOISE
2019-01-01 09:19:18,Suburb,121023,1,2019-01-01,1,TSOL,2019-01-01 09:18:30,1.0,4.0,17.0,Pontoise,Creil,PONTOISE,CREIL
2019-01-01 09:36:29,Paris,121024,1,2019-01-01,1,TOLI,2019-01-01 09:35:50,1.0,0.0,11.0,Creil,Pontoise,CREIL,PONTOISE
2019-01-01 10:36:25,Paris,121026,1,2019-01-01,1,TOLI,2019-01-01 10:35:50,5.0,2.0,24.0,Creil,Pontoise,CREIL,PONTOISE
2019-01-01 10:18:54,Suburb,121027,1,2019-01-01,1,TRIA,2019-01-01 10:18:20,4.0,1.0,26.0,Pontoise,Persan-Beaumont,PONTOISE,PERSAN BEAUMONT


In [17]:
comparison = comparator.Comparator(
          dataset,
          prop_nan, 
          models_to_test, 
          cols_to_impute,
          search_params=search_params,
)
results = comparison.compare()

pp = pprint.PrettyPrinter(depth=4)
pp.pprint(results)

squared_errors load    1290.0
dtype: float64
load    35.91657
dtype: float64
66.0
defaultdict(<class 'list'>, {'rmse': [load    35.91657
dtype: float64], 'mae': [66.0]})


AttributeError: 'DataFrame' object has no attribute 'name'